In [72]:
import numpy as np
import pandas as pd

In [73]:
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict_proba(self, X):
        return np.array([self._predict_proba(x) for x in X.values])

    def _predict_proba(self, x):
        distances = self.compute_distance(x, self.X_train.values)
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = self.y_train.iloc[k_indices]
        return np.mean(k_nearest_labels)

    def compute_distance(self, X1, X2):
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X2 - X1)**2, axis=1))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X2 - X1), axis=1)
        else:
            raise ValueError("Unsupported distance metric")

In [74]:
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Combine train and test for preprocessing
    combined_data = pd.concat([train_data, test_data], axis=0, ignore_index=True)
    
    # Handle categorical variables
    combined_data = pd.get_dummies(combined_data, columns=['Geography', 'Gender'])
    
    # Select relevant features
    features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'] + \
               [col for col in combined_data.columns if col.startswith('Geography_') or col.startswith('Gender_')]
    
    # Scale features
    for feature in features:
        mean = combined_data[feature].mean()
        std = combined_data[feature].std()
        combined_data[feature] = (combined_data[feature] - mean) / std
    
    # Split back into train and test
    X_train = combined_data.loc[:len(train_data)-1, features]
    y_train = train_data['Exited']
    X_test = combined_data.loc[len(train_data):, features]
    
    return X_train, y_train, X_test

In [75]:
def roc_auc_score(y_true, y_pred_proba):
    # Simple implementation of ROC AUC score
    positive_indices = np.where(y_true == 1)[0]
    negative_indices = np.where(y_true == 0)[0]
    
    n_pos = len(positive_indices)
    n_neg = len(negative_indices)
    
    positive_predictions = y_pred_proba[positive_indices]
    negative_predictions = y_pred_proba[negative_indices]
    
    correct_order = sum(p > n for p in positive_predictions for n in negative_predictions)
    
    return correct_order / (n_pos * n_neg)

In [76]:
def cross_validate(X, y, knn, n_splits=5):
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size = len(X) // n_splits
    scores = []
    
    for i in range(n_splits):
        val_indices = indices[i*fold_size:(i+1)*fold_size]
        train_indices = np.concatenate([indices[:i*fold_size], indices[(i+1)*fold_size:]])
        
        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]
        
        knn.fit(X_train, y_train)
        y_pred_proba = knn.predict_proba(X_val)
        score = roc_auc_score(y_val, y_pred_proba)
        scores.append(score)
    
    return scores

In [77]:
# Load and preprocess data
X, y, X_test = preprocess_data('train.csv', 'test.csv')

# Create and evaluate model
knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)
print("Mean ROC AUC score:", np.mean(cv_scores))

# Hyperparameter tuning
k_values = [3, 5, 7, 9, 11, 13, 15, 17]
distance_metrics = ['euclidean', 'manhattan']
best_score = 0
best_params = {}

for k in k_values:
    for metric in distance_metrics:
        knn = KNN(k=k, distance_metric=metric)
        scores = cross_validate(X, y, knn)
        mean_score = np.mean(scores)
        
        if mean_score > best_score:
            best_score = mean_score
            best_params = {'k': k, 'distance_metric': metric}

print("Best parameters:", best_params)
print("Best ROC AUC score:", best_score)

# Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(**best_params)
knn.fit(X, y)
test_predictions = knn.predict_proba(X_test)

rounded_predictions = np.round(test_predictions, 2)

# Save test predictions
pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': rounded_predictions.ravel()}).to_csv('submissions.csv', index=False)

Cross-validation scores: [np.float64(0.7837026455140117), np.float64(0.8193873678503659), np.float64(0.8125610574347233), np.float64(0.7861145617667357), np.float64(0.8205360286978428)]
Mean ROC AUC score: 0.8044603322527358
Best parameters: {'k': 17, 'distance_metric': 'manhattan'}
Best ROC AUC score: 0.8788329671681072
